# Python - Conexión a Base de Datos

Este documento proporciona un ejemplo de cómo usar Python para conectarse a una base de datos PostgreSQL y realizar consultas utilizando diversas bibliotecas.

## Instalación de Paquetes Necesarios
Antes de comenzar, asegúrese de tener instalada la biblioteca psycopg2, que permite la conexión a PostgreSQL desde Python. Si no está instalada, puede hacerlo ejecutando:

In [1]:
#!pip install psycopg2
#!pip install psycopg2-binary

### Importar biblioteca (libreria) psycopg2

In [3]:
import psycopg2
import logging

# Configura el nivel de logging y el formato
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


### Función para Leer Datos de PostgreSQL
A continuación, definimos una función pg_read que se conecta a la base de datos, ejecuta una consulta SQL y devuelve los resultados.

In [7]:
def pg_read(sql_query, conn_str):
    """
    Ejecutar una consulta SQL en una base de datos PostgreSQL y devolver los resultados.

    :param sql_query: La consulta SQL a ejecutar.
    :param conn_str: La cadena de conexión a la base de datos PostgreSQL.
    :return: Los resultados de la consulta SQL.
    """
 
    try:
        with psycopg2.connect(conn_str) as conn:
            with conn.cursor() as cursor:
                cursor.execute(sql_query)
                res = cursor.fetchall()
                print('La consulta SQL se ejecutó en la base de datos PostgreSQL y los resultados se han recuperado.')
                return res
           
    except Exception as e:
        logging.error(f'Un error ocurrió: {e}')
       
    finally:
        conn.close()

### Parámetros de Conexión a la Base de Datos
Definimos los parámetros necesarios para conectarnos a la base de datos PostgreSQL:

In [9]:
# Parámetros de conexión
usuario = 'postgres'
contraseña = 'postgre' # Cambia esto por tu contraseña real
host = 'localhost'
puerto = '5432'  # Puerto por defecto de PostgreSQL
base_de_datos = 'postgres'

# String de conexión
conn_str = f"host={host} port={puerto} dbname={base_de_datos} user={usuario} password={contraseña}"

A continuación les dejo un código de ejemplo que les permitirá comprobar la conectividad a la base de datos asi como obtener información del servidor de PostgreSQL

In [11]:
try:
    with psycopg2.connect(conn_str) as connection:
        print("Conexión exitosa.")
        with connection.cursor() as cursor:
            cursor.execute("SELECT version();")
            version = cursor.fetchone()
            print(f"Versión del servidor PostgreSQL: {version[0]}")
except Exception as e:
    print(f"Ocurrió un error al conectar a la base de datos: {e}")

Conexión exitosa.
Versión del servidor PostgreSQL: PostgreSQL 17.5 on x86_64-windows, compiled by msvc-19.43.34808, 64-bit


### Ejecución de Consultas SQL
Ejemplo de cómo usar la función pg_read para ejecutar una consulta SQL y obtener resultados:

In [13]:
pg_read("SELECT * FROM dw_tfm_restaurantes.restaurantes", conn_str)

La consulta SQL se ejecutó en la base de datos PostgreSQL y los resultados se han recuperado.


[]

### Consulta para Información de Columnas de una Tabla
Podemos usar la misma función pg_read para ejecutar consultas más específicas, como obtener información sobre las columnas de una tabla particular en la base de datos.

In [15]:
query = """
SELECT 
  column_name, 
  data_type, 
  character_maximum_length, 
  is_nullable, 
  column_default 
FROM 
  information_schema.columns 
WHERE 
  table_name = 'restaurantes';
"""

In [17]:
pg_read(query, conn_str)

La consulta SQL se ejecutó en la base de datos PostgreSQL y los resultados se han recuperado.


[('id_local', 'bigint', None, 'NO', None),
 ('id_distrito_local', 'integer', None, 'YES', None),
 ('desc_distrito_local', 'character varying', 255, 'YES', None),
 ('id_barrio_local', 'integer', None, 'YES', None),
 ('desc_barrio_local', 'character varying', 255, 'YES', None),
 ('coordenada_x_local', 'double precision', None, 'YES', None),
 ('coordenada_y_local', 'double precision', None, 'YES', None),
 ('latitud', 'double precision', None, 'YES', None),
 ('longitud', 'double precision', None, 'YES', None),
 ('rotulo', 'character varying', 255, 'YES', None),
 ('id_seccion', 'character varying', 50, 'YES', None),
 ('desc_seccion', 'character varying', 255, 'YES', None),
 ('id_division', 'integer', None, 'YES', None),
 ('desc_division', 'character varying', 255, 'YES', None),
 ('rotulo_new', 'character varying', 255, 'YES', None),
 ('tipo_cocina', 'character varying', 100, 'YES', None),
 ('revisar_manual', 'character varying', 50, 'YES', None)]

## Uso de SQLAlchemy para Conexión a la Base de Datos
SQLAlchemy es otra biblioteca poderosa para manejar bases de datos en Python. Permite una mayor abstracción y facilita algunas operaciones.

### Instalación de SQLAlchemy
Si aún no está instalada, puede instalar la biblioteca SQLAlchemy utilizando:

In [10]:
#!pip install sqlalchemy

### Conexión a PostgreSQL Usando SQLAlchemy
Primero, importamos create_engine de SQLAlchemy y establecemos los parámetros de conexión:

In [19]:
from sqlalchemy import create_engine, text

# Parámetros de conexión
usuario = 'postgres'
contraseña = 'postgre'
host = 'localhost'
puerto = '5432'  # Puerto por defecto de PostgreSQL
base_de_datos = 'postgres'

# String de conexión
conexion = f'postgresql+psycopg2://{usuario}:{contraseña}@{host}:{puerto}/{base_de_datos}'

# Crear el motor de SQLAlchemy
engine = create_engine(conexion)

### Probando la Conexión
Aquí, intentamos conectarnos a la base de datos y realizar una operación simple para probar la conexión:

In [21]:
try:
    with engine.connect() as connection:
        print("Conexión exitosa.")
except Exception as e:
    print(f"Ocurrió un error al conectar a la base de datos: {e}")

Conexión exitosa.


### Consulta SQL para Obtener las Tablas del Esquema
Usamos la conexión establecida para ejecutar una consulta que nos devuelva los nombres de las tablas en un esquema específico de la base de datos:

In [23]:
consulta = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'dw_tfm_restaurantes'
ORDER BY table_name;
"""

# Ejecutar la consulta
try:
    with engine.connect() as connection:
        result = connection.execute(text(consulta))
        for row in result.mappings():
            print(row['table_name'])
except Exception as e:
    print(f"Ocurrió un error al realizar la consulta: {e}")

aparcamientos
dim_barrio
dim_distrito
flujo_peatones
licencias
menu_restaurantes_mx
poblacion_madrid
residentes_demografia
restaurantes
terrazas
transporte_publico


## Uso de Pandas para Análisis de Datos
Pandas es una biblioteca de Python muy popular para el análisis de datos. Permite leer datos de diversas fuentes, incluidas bases de datos SQL, y trabajar con ellos de manera eficiente.

### Importar Pandas
Primero, debemos importar Pandas:

In [25]:
import pandas as pd

### Cargar Datos en un DataFrame de Pandas desde una BBDD
Podemos usar Pandas para leer datos directamente desde la base de datos a un DataFrame.

#### Leer una Tabla Completa

In [27]:
pd.read_sql_table('restaurantes', con=conexion, schema='dw_tfm_restaurantes')

,id_local,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,coordenada_x_local,coordenada_y_local,latitud,longitud,rotulo,id_seccion,desc_seccion,id_division,desc_division,rotulo_new,tipo_cocina,revisar_manual


#### Ejecutar una Consulta SQL y Leer los Resultados

In [29]:
pd.read_sql_query('select * from dw_tfm_restaurantes.restaurantes limit 10', con=conexion)

,id_local,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,coordenada_x_local,coordenada_y_local,latitud,longitud,rotulo,id_seccion,desc_seccion,id_division,desc_division,rotulo_new,tipo_cocina,revisar_manual


### Cargar Datos de un DataFrame de Pandas a una Base de Datos SQL
Una característica poderosa de Pandas es su capacidad para cargar datos directamente desde un DataFrame a una base de datos SQL. Esto se hace usando el método .to_sql() del DataFrame

In [58]:
df.to_sql(name, con, if_exists='fail', index=True)

NameError: name 'df' is not defined

__name:__ Nombre de la tabla SQL donde se cargarán los datos.  
__con:__ Objeto de conexión SQLAlchemy a la base de datos.  
**if_exists:** Qué hacer si la tabla ya existe. Las opciones son 'fail', 'replace' y 'append'.  
__index:__ Si es True, el índice del DataFrame se insertará como una columna en la tabla. Si es False, no se incluirá.  